In [1]:
%matplotlib inline
from ipywidgets import interactive
import matplotlib.pyplot as plt
import numpy as np

In [2]:

import csv

import matplotlib.pyplot as plt
import numpy as np
from astropy.convolution import Gaussian1DKernel, convolve
from scipy.optimize import least_squares
from joblib import Parallel, delayed
from kaufmann import *

path_to_file = 'hT__Heterostructure_5keV_BGO.txt'
counts = []
with open(path_to_file, 'r') as myfile:
    csv_reader = csv.reader(
        myfile,
        delimiter=' ',
        skipinitialspace=True,
        escapechar='#',
        quoting=csv.QUOTE_NONNUMERIC)
    for number, line in enumerate(csv_reader):
        counts = np.append(counts, line[0])

# range_end = 6001
norm = 15 * 10**3
background = np.mean(counts[5000:5900])
posC0 = 399
sigma = 10
# changed (kevin)
X_train = np.linspace(1, 5000, 5000)
Y_train = counts[1:5000 + 1]


def Jacobian(x, params):
    # wolfram alpha:
    # jacobian (a_0, a_1*exp(-b_1*x), a_2*exp(-b_2*x),a_3*exp(-b_3*x),a_4*exp(-b_4*x)) with respect to (a_0, a_1, a_2, a_3, a_4, b_1, b_2, b_3, b_4)
    a = params[0:5]
    b = params[5:]
    J = np.array([
         [
             1, 0, 0, 0, 0, 0, 0, 0, 0
         ],
         [
          0, np.exp(-x * b[1]), 0, 0, 0, -np.exp(-x * b[1]) * x * a[1], 0, 0, 0
         ],
         [
          0, 0, np.exp(-x * b[2]), 0, 0, 0, -np.exp(-x * b[2]) * x * a[2], 0, 0
         ],
         [
          0, 0, 0, np.exp(-x * b[3]), 0, 0, 0, -np.exp(-x * b[3]) * x * a[3], 0
         ],
         [
          0, 0, 0, 0, np.exp(-x * b[4]), 0, 0, 0, -np.exp(-x * b[4]) * x * a[4]
         ]
                 ])
    return J


In [8]:
X_train = X_train[397:2000]
Y_train = Y_train[397:2000] / Y_train[397:].max()


def exp_decay(e, c, d):
    array = []
    a, b = Kaufmann2003Solve(int(4), X_train, Y_train)
    # range end is now an integer
    for x in range(len(X_train)):
        value = a[0] + e * np.exp(-b[0]*x)   \
                     + c * np.exp(-b[1]*x)   \
                     + d * np.exp(-b[2]*x)   \
                     + a[4] * np.exp(-b[3]*x)
        array = np.append(array, value)
    gauss_1D_kernel_250ps = Gaussian1DKernel(sigma)
    astropy_conv = convolve(array, gauss_1D_kernel_250ps)
    Y_train_fit = astropy_conv / astropy_conv.max()
    plt.semilogy(X_train, Y_train_fit, label='fit')
    plt.semilogy(X_train, Y_train, label = 'real data')
    plt.xlabel('250ps/bin')
    plt.ylabel('counts')
    plt.legend(loc = 'best')
    plt.title('random fit')
    plt.show()

In [11]:
interactive_plot = interactive(exp_decay, e=(0, 2.0, 0.1), c=(-0.1, 3, 0.5), d = (0, 3, 0.5))
output = interactive_plot.children[-1]
output.layout.height = '350px'
interactive_plot

interactive(children=(FloatSlider(value=1.0, description='e', max=2.0), FloatSlider(value=1.4, description='c'…